# Estudio completo de un problema de ML con Python
### Estimación del precio de una vivienda 

En este notebook haremos un análisis exploratorio básico de la base de datos de viviendas [House Sales in King County, USA](https://www.kaggle.com/harlfoxem/housesalesprediction), para familiarizarnos con los datos y posteriormente aplicar técnicas de machine learning sobre ellos. 

Lo primero es echar un ojo a la fuente; en este caso es Kaggle, por lo que disponéis de muchísima información sobre los datos (no os acostumbréis).

Sabemos que para cada vivienda, se tienen los siguientes atributos, características o features:

| Atributo | descripción |
| :- |:- |
|*id*| identificador de la vivienda|
| *date*| fecha
| *price*| precio
| *bedrooms*| número de habitaciones
| *bathrooms*| número de baños/aseos
| *sqtf_living*| superficie habitable (en pies al cuadrado)
| *sqft_lot*| superficie de la parcela (en pies al cuadrado)
| *floors*| número de plantas
| *waterfront*| indica si la vivienda tiene acceso a un lago
| *view*| tipo de vista (variable numérica)
| *condition*| condición de la vivienda (variable númerica)
| *grade*| medida de la calidad de la construcción (variable numérica)
| *sqft_above*| superficie por encima del suelo (en pies al cuadrado)
| *sqft_basement*| superficie del sótano (en pies al cuadrado)
| *yr_built*| año de construcción de la vivienda
| *yr_renovated*| año de renovación de la vivienda
| *lat*| latitud de la parcela
| *long*| longitud de la parcela
| *sqft_living15*| superficie habitable promedio de los 15 vecinos más cercanos 				
| *sqft_lot15*| superficie de la parcela promedio de los 15 vecinos más cercanos

Vamos a utilizar **DataFrames** de [Pandas](http://pandas.pydata.org/). Como es sabido, Pandas es un módulo de python de código abierto para el análisis de datos, que proporciona estructuras de datos fáciles de utilizar. Como guía de referencia básica, puede consultarse la [cheat sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)

In [ ]:
import numpy  as np  
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

## 1. Carga de datos y división train/test

Hay que tener mucho cuidado a la hora de realizar la división, para no cometer data leakage. Yo recomiendo que echéis un ojo al dataset, eliminéis todas aquellas columnas que sabéis que podéis quitar gracias a vuestro conocimiento del dominio (ids, URLs, etc) y a continuación dividáis en train/test para evitar riesgos.

In [ ]:
house_data = pd.read_csv("./data/king_county.csv") # cargamos fichero
print(house_data.shape)
house_data.head(5).T                                 # visualizamos 5 primeras filas

El método `train_test_split` es mucho más potente que lo que hemos visto hasta ahora en clase, y permite hacer muchas más cosas. De momento siempre hemos particionado en cuatro (xtrain, xtest, ytrain, ytest) obteniendo arrays de numpy, pero no es la única opción. Aquí tenéis un único fichero .csv, y podéis usar la función para obtener dos subconjuntos: train y test.

In [ ]:
from sklearn.model_selection import train_test_split

full_df = pd.read_csv("./data/king_county.csv")
train, test = train_test_split(full_df, test_size=0.2, shuffle=True, random_state=0)

print(f'Dimensiones del dataset de training: {train.shape}')
print(f'Dimensiones del dataset de test: {test.shape}')

# Guardamos
train.to_csv('./data/king_county_train.csv', sep=';', decimal='.', index=False)
test.to_csv('./data/king_county_test.csv', sep=';', decimal='.', index=False)

# A partir de este momento cargamos el dataset de train y trabajamos ÚNICAMENTE con él. 

house_data = pd.read_csv('./data/king_county_train.csv', sep=';', decimal='.')
house_data.head(5).T

## 2. Análisis exploratorio

Podemos analizar la estructura básica del dataset con las funciones de Pandas que ya conocemos: `describe`, `dtypes`, `shape`, etc.

In [ ]:
house_data.describe()

<div class = "alert alert-success">
EJERCICIO 3.6: Aunque no es fácil de ver, hay un outlier *extremo* ahí arriba. ¿Cuál es?
</div>

In [ ]:
house_data.isnull().any()

#### Imputación

Vemos que la variable `bedrooms` tiene valores ausentes; hay que imputar. Es muy sencillo con pandas, usando fillna:

`df["Feature"].fillna(df["Feature"].mode()[0], inplace=True)`

Se puede rellenar con la moda (valor más frecuente), en otras ocasiones es preferible usar la media, y en algunos (pocos) casos se puede hacer con ceros.

In [ ]:
house_data['bedrooms'].fillna(house_data['bedrooms'].mode()[0], inplace=True)
house_data.isnull().any()

Vamos a ver qué es esa variable `date`, no encaja demasiado con el resto del dataset:

In [ ]:
# Podemos ordenar un dataframe con esta sintaxis:
sorted_df = house_data.sort_values(by='date')
sorted_df['date'].head(10)

In [ ]:
# Y para orden inverso:
sorted_df = house_data.sort_values(by='date', ascending=False)
sorted_df['date'].head(10)

Vemos que la columna va de 2014 a 2015, y probablemente haga referencia a la fecha de venta. No interesa.

In [ ]:
house_data.dtypes

#### Codificación de variables categóricas

Podemos observar que todas las variables son de tipo numérico, así que no tenemos que codificar ninguna de ellas. Pero... y si necesitáramos hacerlo? 

[MeanEncoder](https://maxhalford.github.io/blog/target-encoding/): en sklearn se llama [TargetEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.TargetEncoder.html). Asigna un valor a cada variable categórica según la media de la columna objetivo para el conjunto de registros que tienen esa variable categórica. Es decir, si quisiera categorizar la variable "Barrio" con un ME, lo que tendría que hacer es calcular la media de precio en cada barrio (Villaverde, Chamberí, etc) y sustituir el nombre del barrio por esa media. Ojito con el data leakage, os dejo un ejemplo debajo de cómo hacerlo bien.

Tutoriales sobre codificación de variables categóricas: [Tutorial 1](https://towardsdatascience.com/encoding-categorical-features-21a2651a065c), [tutorial 2](https://towardsdatascience.com/all-about-categorical-variable-encoding-305f3361fd02)

In [ ]:
# Target encoder hace algo así por debajo:

"""
categorical = ['cat1', 'cat2', 'cat3']

mean_map = {}
for c in categorical:
    mean = data.groupby(c)['target'].mean()
    data[c] = data[c].map(mean)    
    mean_map[c] = mean

# Si hubiera test, luego se haría:
#for c in categorical:
#    data_test[c] = data_test[c].map(mean_map[c])

data.head()
"""

Este es uno de los tutoriales más completos que he visto: [Encoding done the right way](https://maxhalford.github.io/blog/target-encoding/) y me hace especial gracia porque dice esto: "Label encoding is useless and you should never use it". No estoy 100% de acuerdo obviamente :) pero sí que es cierto que cuando hay muchas categorías (es decir, no es binario) un LE puede llevar a errores porque asigna números a cada una de ellas, con lo cual el algoritmo puede "aprender" erróneamente. Supongamos que tengo una categoría barrio que quiero categorizar:

- Barrio céntrico moderno y caro -> LE -> 1
- Otro barrio -> LE -> 2
- Otro más -> LE -> 3
- Y otro -> LE -> 4
- Barrio periférico y obrero -> LE -> 5

Mis categorías tras el LE pasarían a ser 1-5, pero qué quiere decir esto? Que 5 es mayor que 1? Que 3 es menor que 5? No, porque no existe esa relación entre los barrios, pero SÍ entre los números! Entonces el algoritmo podría decidir que de alguna manera "Barrio periférico y obrero > Barrio céntrico moderno y caro" porque 5 > 1.

Otro encoder que podéis usar y que va bastante bien es el [OrdinalEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html).

----------------------------------------

Volviendo al análisis, los atributos *id* y *date* no nos aportan información, así que los descartamos del DataFrame:

In [ ]:
# Eliminamos las columnas id y date 
house_data = house_data.drop(['id','date'], axis=1)
house_data.head(5).T

Antes seguir con nuestro análisis, vamos a transformar las variables de superficie para expresarlas en $m^2$. Posteriormente, renombraremos las columnas.

In [ ]:
pies = [item for item in list(house_data.columns) if 'sqft' in item]
pies

In [ ]:
# convertir las variables en pies al cuadrado en metros al cuadrado 
feetFeatures = ['sqft_living','sqft_lot','sqft_above','sqft_basement','sqft_living15','sqft_lot15']

def sqft_to_m2(superficie):
    return superficie * 0.3048 * 0.3048

house_data[feetFeatures] = house_data[feetFeatures].apply(sqft_to_m2)

# Alternativa usando una función lambda
# house_data[feetFeatures] = house_data[feetFeatures].apply(lambda x: x * 0.3048 * 0.3048)

# renombramos
house_data.columns = ['price','bedrooms','bathrooms','sqm_living','sqm_lot','floors','waterfront','view','condition',
                      'grade','sqm_above','sqm_basement','yr_built','yr_renovated','zip_code','lat','long',
                      'sqm_living15','sqm_lot15']

# visualizamos
house_data.head(5)

## 3. Visualización (y más análisis)

Una buena práctica es intentar resumir toda la información posible de los datos. Habitualmente nos interesa saber la media y desviación estándar, posiblemente cuartiles de cada una de las variables. Esto nos permitirá, por una lado, tener una idea de cómo son las ditribuciones de cada una de las variables y por otra, nos permitirá verificar si existen datos anómalos, también conocidos como [**outliers**](https://en.wikipedia.org/wiki/Outlier). 

Además, conviene siempre hacer representaciones gráficas, que nos ofrecen, en general un mejor entendimiento de los datos. Para ello vamos representar los histogramas de algunos atributos: *bedrooms*, *sqm_living* y *yr_built*

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1,3,1)
house_data['bedrooms'].plot.hist(alpha=0.5, bins=25, grid = True)
#plt.axis([0, 10, 0, 10000])
plt.ylim(0, 15)
plt.xlabel('bedrooms')

plt.subplot(1,3,2)
house_data['sqm_living'].plot.hist(alpha=0.5, bins=25, grid = True)
plt.yscale("log")
plt.xlabel('sqm_living')

plt.subplot(1,3,3)
house_data['yr_built'].plot.hist(alpha=0.5, bins=25, grid = True)
plt.xlabel('yr_built')

plt.show()

Una vez que hemos analizado las variables por separado, el siguiente paso en un análisis exploratorio sería el entender las relaciones entre cada una de las variables/atributos ($\mathbf{x}$) y la variable respuesta ($y$). 

Para ello vamos a utilizar un [scatter plot](https://en.wikipedia.org/wiki/Scatter_plot) con la variable objetivo definida $y$ como variable dependiente, y alguna una de las variables explicativas como variables independientes. En el caso de la variable *waterfront*, dado que ésta es binaria, vamos a utilizar un [boxplot](https://en.wikipedia.org/wiki/Box_plot).

In [ ]:
# Sólo representamos 3: bedrooms, sqm_living y waterfront
# el resto se puede repetir una a una

house_data.plot(kind = 'scatter',x='bedrooms',y = 'price')
plt.xlabel('# bedrooms')
plt.ylabel('price ($)')
plt.show()

house_data.plot(kind = 'scatter',x='sqm_living',y = 'price')
plt.xlabel('sqm_living ($m^2$)')
plt.ylabel('price ($)')
plt.show()

house_data.boxplot(by='waterfront',column = 'price')
plt.show()

Podemos estudiar algunas de las variables con `.value_counts()`, por ejemplo *waterfront* o *bedrooms*.

<div class = "alert alert-success">
EJERCICIO 3.7: Analizar los `value_counts` de *waterfront* y *bedrooms*
</div>

#### Eliminación de outliers

Tanto con los scatter plot de arriba como con los análisis de `value_counts`, vemos que hay unos pocos outliers en la variable bedrooms (y en sqm_living). Vamos a eliminarlos con un filtro. Si recordáis, vimos en el repaso de Pandas que los dataframes podían filtrarse con `df_filtered = df[condición]`:

In [ ]:
house_data_no_outliers_bedrooms = house_data[house_data['bedrooms'] <= 8]

house_data_no_outliers_bedrooms.plot(kind = 'scatter',x='bedrooms',y = 'price')
plt.xlabel('# bedrooms')
plt.ylabel('price ($)')
plt.show()

¿Hemos perdido muchos datos con la eliminación?

In [ ]:
print(
    f'Original: {house_data.shape[0]} // '
    f'Modificado: {house_data_no_outliers_bedrooms.shape[0]}\nDiferencia: {house_data.shape[0] - house_data_no_outliers_bedrooms.shape[0]}'
)
print(f'Variación: {((house_data.shape[0] - house_data_no_outliers_bedrooms.shape[0])/house_data.shape[0])*100:2f}%')

<div class = "alert alert-success">
EJERCICIO 3.8: Repetir el estudio de outliers para la variable *sqm_living*
</div>

Una vez que hemos hecho un primer análisis exploratorio, el siguiente paso consiste en evaluar las correlaciones entre las diferente variables del problema. Habitualmente, esto nos puede servir para identificar posibles atributos que estén altamente correlacionados. 

Si la correlación entre dos atributos es muy grande, se dice que la matriz de atributos es singular, y como ya vimos, esto es una fuente de error importante en algunos algoritmos de machine learning, como por ejemplo en el caso de la [regresión lineal](https://es.wikipedia.org/wiki/Regresión_lineal). 

Este problema se denomina *colinealidad*. Para hacer frente a él, normalmente se evalúa [coeficiente de correlación](https://es.wikipedia.org/wiki/Coeficiente_de_correlación_de_Pearson) ($\rho$) entre las diferentes atributos de tal forma que se descartan que tengan un $\rho$ superior a un umbral que establezcamos a priori ($|\rho|>0.9$, por ejemplo). Hay que tener en cuenta que $-1<\rho<1$, de tal forma que valores próximos a $0$ indican que no hay correlación y valores próximos a $1$ o $-1$ indican una alta correlación.

La matriz de correlación se puede sacar de Pandas:

In [ ]:
house_data.corr() # matriz de correlación

Pero es mejor representarla:

In [ ]:
import seaborn as sns

# Compute the correlation matrix
corr = np.abs(house_data.drop(['price'], axis=1).corr())

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(12, 10))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask,vmin = 0.0, vmax=1.0, center=0.5,
            linewidths=.1, cmap="YlGnBu", cbar_kws={"shrink": .8})

plt.show()

Basándonos en el estudio podríamos llegar a valorar eliminar la variable *sqm_above*.

Por último, podemos hacer una representación (scatter_plot) de todas las variables frente al resto, para tener una idea de cómo se relacionan las variables del problema.

In [ ]:
pd.plotting.scatter_matrix(house_data, alpha=0.2, figsize=(20, 20), diagonal = 'kde')
plt.show()

## 4. Generación de nuevas características

Este sería el momento de pensar sobre otras variables que tuvieran sentido. Por ejemplo:

- Construir el atributo antigüedad de la casa en vez de año de la construcción.
- Intentar capturar la relación entre dormitorios y baños
- Cualquier otra idea que tengáis. Por ejemplo, para este dataset en concreto la gente encontró que elevar al cuadrado el número de dormitorios era una variable relevante; la explicación (o eso se cree) es que muchas de estas casas habían subdividido habitaciones para poder introducir a más inquilinos, a veces incluso a costa de las zonas comunes. Por tanto, un número de dormitorios mayor era mejor _hasta cierto punto_, lo cual quedaba recogido al elevar al cuadrado. Fijaos como en el scatter plot de bedroom podemos ver esta misma tendencia.

In [ ]:
house_data['years']            = 2015 - house_data['yr_built']
house_data['bedrooms_squared'] = house_data['bedrooms'].apply(lambda x: x**2)
house_data['bed_bath_rooms']   = house_data['bedrooms']*house_data['bathrooms']

# ... etc

## 5. Modelado, cross-validation y estudio de resultados en train y test

Ha llegado el gran momento! Antes de modelar, tenemos que cargar los datos de test y aplicar exactamente las mismas transformaciones. Es buena práctica, llegado este momento, combinar todo nuestro preprocesamiento en una única celda:

In [ ]:
# Carga de datos
house_data = pd.read_csv('./data/king_county_train.csv', sep=';', decimal='.')

# Imputación
house_data['bedrooms'].fillna(house_data['bedrooms'].mode()[0], inplace=True)

# Eliminamos las columnas id y date 
house_data = house_data.drop(['id','date'], axis=1)

# Convertimos las variables en pies al cuadrado en metros al cuadrado y renombramos
feetFeatures = ['sqft_living','sqft_lot','sqft_above','sqft_basement','sqft_living15','sqft_lot15']

def sqft_to_m2(superficie):
    return superficie * 0.3048 * 0.3048

house_data[feetFeatures] = house_data[feetFeatures].apply(sqft_to_m2)
house_data.columns = ['price','bedrooms','bathrooms','sqm_living','sqm_lot','floors','waterfront','view','condition',
                      'grade','sqm_above','sqm_basement','yr_built','yr_renovated','zip_code','lat','long',
                      'sqm_living15','sqm_lot15']

# Eliminamos outliers en bedrooms
house_data = house_data[house_data['bedrooms'] <= 8]

# Generamos características
house_data['years']            = 2015 - house_data['yr_built']
house_data['bedrooms_squared'] = house_data['bedrooms'].apply(lambda x: x**2)
house_data['bed_bath_rooms']   = house_data['bedrooms']*house_data['bathrooms']

Y ahora aplicamos fácilmente a test:

In [ ]:
# Carga de datos
house_data_test = pd.read_csv('./data/king_county_test.csv', sep=';', decimal='.')

# Imputación
house_data_test['bedrooms'].fillna(house_data_test['bedrooms'].mode()[0], inplace=True)

# Eliminamos las columnas id y date 
house_data_test = house_data_test.drop(['id','date'], axis=1)

# Convertimos las variables en pies al cuadrado en metros al cuadrado y renombramos
feetFeatures = ['sqft_living','sqft_lot','sqft_above','sqft_basement','sqft_living15','sqft_lot15']

def sqft_to_m2(superficie):
    return superficie * 0.3048 * 0.3048

house_data_test[feetFeatures] = house_data_test[feetFeatures].apply(sqft_to_m2)
house_data_test.columns = ['price','bedrooms','bathrooms','sqm_living','sqm_lot','floors','waterfront','view','condition',
                      'grade','sqm_above','sqm_basement','yr_built','yr_renovated','zip_code','lat','long',
                      'sqm_living15','sqm_lot15']

# Eliminamos outliers en bedrooms
house_data_test = house_data_test[house_data_test['bedrooms'] <= 8]

# Generamos características
house_data_test['years']            = 2015 - house_data_test['yr_built']
house_data_test['bedrooms_squared'] = house_data_test['bedrooms'].apply(lambda x: x**2)
house_data_test['bed_bath_rooms']   = house_data_test['bedrooms']*house_data_test['bathrooms']

Ahora podemos preparar los datos para sklearn:

In [ ]:
from sklearn import preprocessing

# Dataset de train
data_train = house_data.values
y_train = data_train[:,0:1]     # nos quedamos con la 1ª columna, price
X_train = data_train[:,1:]      # nos quedamos con el resto

# Dataset de test
data_test = house_data_test.values
y_test = data_test[:,0:1]     # nos quedamos con la 1ª columna, price
X_test = data_test[:,1:]      # nos quedamos con el resto

Y si queremos, podemos normalizar, pero con los datos de train!

In [ ]:
# Escalamos (con los datos de train)
scaler = preprocessing.StandardScaler().fit(X_train)
XtrainScaled = scaler.transform(X_train)

# recordad que esta normalización/escalado la realizo con el scaler anterior, basado en los datos de training!
XtestScaled = scaler.transform(X_test) 

In [ ]:
print('Datos entrenamiento: ', XtrainScaled.shape)
print('Datos test: ', XtestScaled.shape)

Ahora vendría lo-de-siempre: cross validation, búsqueda de los parámetros óptimos, visualización de performance vs complejidad...

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

alpha_vector = np.logspace(-1,10,20)
param_grid = {'alpha': alpha_vector }
grid = GridSearchCV(Lasso(), scoring= 'neg_mean_squared_error', param_grid=param_grid, cv = 3, verbose=2)
grid.fit(XtrainScaled, y_train)
print("best mean cross-validation score: {:.3f}".format(grid.best_score_))
print("best parameters: {}".format(grid.best_params_))

#-1 porque es negado
scores = -1*np.array(grid.cv_results_['mean_test_score'])
plt.semilogx(alpha_vector,scores,'-o')
plt.xlabel('alpha',fontsize=16)
plt.ylabel('3-Fold MSE')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error

alpha_optimo = grid.best_params_['alpha']
lasso = Lasso(alpha = alpha_optimo).fit(XtrainScaled,y_train)

ytrainLasso = lasso.predict(XtrainScaled)
ytestLasso  = lasso.predict(XtestScaled)
mseTrainModelLasso = mean_squared_error(y_train,ytrainLasso)
mseTestModelLasso = mean_squared_error(y_test,ytestLasso)

print('MSE Modelo Lasso (train): %0.3g' % mseTrainModelLasso)
print('MSE Modelo Lasso (test) : %0.3g' % mseTestModelLasso)

print('RMSE Modelo Lasso (train): %0.3g' % np.sqrt(mseTrainModelLasso))
print('RMSE Modelo Lasso (test) : %0.3g' % np.sqrt(mseTestModelLasso))

feature_names = house_data.columns[1:] # es igual en train y en test

w = lasso.coef_
for f,wi in zip(feature_names,w):
    print(f,wi)

Y a partir de aquí, habría que iterar. Los coeficientes son muy grandes; quizá queramos regularizar. O probar otros modelos. O reducir características. O...